In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
# from collections import defaultdict
# import sys; sys.path.append('../')
# from src.data_analysis.tools import ac_pca
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import random 
pd.set_option('display.max_columns', 5000)

In [2]:
# combined_data = pd.read_csv('../data/processed/pre_pca.csv')
combined_data = pd.read_csv('../data/processed/local_pairwise_multi_var_scaled.csv')
# print(combined_data[combined_data['Rural-urban_Continuum Code_2013'] > 2])
# combined_data = combined_data[combined_data['normalized_deaths'] < 0.001]
# print( combined_data[combined_data['normalized_deaths'] > 0.0003].head())
combined_data = combined_data.drop(columns='POP_ESTIMATE_2018')
# combined_data = combined_data[combined_data['Rural-urban_Continuum Code_2013'] > 1]
# combined_data.set_index('FIPS', inplace=True)
data = combined_data.drop(columns=['FIPS1', 'FIPS2', 'Deaths', 'normalized_deaths'])
print(combined_data.shape)
combined_data.head()

(583, 34)


,FIPS1,FIPS2,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,Area in square miles - Land area,NursingCt,HospCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths
0,1003.0,1097.0,-0.342697,0.204549,1.251452,0.907112,1.251452,2.145553,1.423420,2.145553,0.027059,1.068392,-0.757302,0.647714,-0.794326,0.992035,0.793988,0.059330,-0.649475,-1.226761,0.169463,-0.202892,-0.539651,0.206955,0.595805,-0.727074,0.237995,-2.055380,-0.057589,-1.367917,-1.050986,1.294704,-33.0,-0.000067
1,1003.0,12033.0,1.072502,0.904665,2.335637,1.674980,2.335637,2.562506,1.785262,2.562506,-0.001238,1.068392,0.126827,0.378963,0.022775,0.859024,0.674572,0.276417,-0.136818,-1.767844,1.053969,0.215399,-1.135186,0.524709,0.364625,-1.339735,-0.018766,-1.078738,-0.057589,-0.681604,0.021852,1.294704,-5.0,-0.000007
2,1051.0,1101.0,1.674893,2.880798,3.053564,2.699576,3.053564,5.265570,5.002506,5.265570,0.040786,-0.001836,-0.123036,-0.580859,-0.453867,0.772567,0.004968,0.312598,0.613842,-1.564061,-0.762023,0.621988,-0.412909,-0.084807,1.486582,-2.046079,0.043495,-2.844225,1.178964,-1.825459,0.021852,2.839259,1.0,0.000059
3,1125.0,1073.0,0.551483,1.107270,-1.651463,-1.604922,-1.651463,-1.794419,-1.643833,-1.794419,0.187570,-2.142291,-0.219137,0.163963,-0.045317,0.220574,0.484115,-0.555751,0.243573,0.239223,0.465290,0.354604,-1.961163,-0.109596,-1.075709,-1.831544,0.099200,0.865778,-1.456517,2.292418,1.094691,-3.101339,19.0,-0.000010
4,1081.0,13215.0,2.049380,3.173286,1.368273,1.296207,1.368273,1.461298,1.808261,1.461298,-0.075388,1.068392,-0.584320,0.609321,-1.066693,0.676135,-0.277584,0.348780,-0.422970,-1.027793,-0.779063,-0.123655,2.247777,0.224041,0.220790,-2.633390,-0.272957,-1.870864,-0.170003,0.385994,0.826481,-0.131040,15.0,0.000098


In [3]:
def process_likelihood(data):
    pca = PCA(0.99, whiten=True)
    X = pca.fit_transform(data)
    num_components = X.shape[1]
    X = np.append(X,np.array(combined_data['HospCt']).reshape(-1,1), axis=1)
    X = np.append(X,np.array(combined_data['normalized_deaths']).reshape(-1,1), axis=1)
    columns = ['Principal_Component_' + str(i+1) for i in range(num_components)]
    column_names = ''
    for col in columns:
        column_names += col + '+'
    column_names = column_names[0:len(column_names)-1]
    print(column_names)
    print(columns)
    columns.append("HospCt")
    columns.append('normalized_deaths')
    pre_likelihood_ratio_test = pd.DataFrame(data=X, columns=columns)
    return pre_likelihood_ratio_test

In [4]:
with_hosp = process_likelihood(data)
with_hosp.to_csv('../data/processed/pca_withHosp_local_pairwise_multi_var.csv', index=False)
with_hosp.head()

Principal_Component_1+Principal_Component_2+Principal_Component_3+Principal_Component_4+Principal_Component_5+Principal_Component_6+Principal_Component_7+Principal_Component_8+Principal_Component_9+Principal_Component_10+Principal_Component_11+Principal_Component_12+Principal_Component_13+Principal_Component_14+Principal_Component_15+Principal_Component_16+Principal_Component_17+Principal_Component_18+Principal_Component_19+Principal_Component_20+Principal_Component_21+Principal_Component_22
['Principal_Component_1', 'Principal_Component_2', 'Principal_Component_3', 'Principal_Component_4', 'Principal_Component_5', 'Principal_Component_6', 'Principal_Component_7', 'Principal_Component_8', 'Principal_Component_9', 'Principal_Component_10', 'Principal_Component_11', 'Principal_Component_12', 'Principal_Component_13', 'Principal_Component_14', 'Principal_Component_15', 'Principal_Component_16', 'Principal_Component_17', 'Principal_Component_18', 'Principal_Component_19', 'Principal_Compon

,Principal_Component_1,Principal_Component_2,Principal_Component_3,Principal_Component_4,Principal_Component_5,Principal_Component_6,Principal_Component_7,Principal_Component_8,Principal_Component_9,Principal_Component_10,Principal_Component_11,Principal_Component_12,Principal_Component_13,Principal_Component_14,Principal_Component_15,Principal_Component_16,Principal_Component_17,Principal_Component_18,Principal_Component_19,Principal_Component_20,Principal_Component_21,Principal_Component_22,HospCt,normalized_deaths
0,1.444754,-0.772578,-0.881351,0.497203,-0.300229,-0.353880,-1.178197,-1.010186,0.947155,-0.050667,1.415334,0.086548,0.857552,0.430658,-0.854242,-0.292531,-0.479651,-0.279970,1.036788,2.621792,0.123348,-0.616840,-0.727074,-0.000067
1,2.019263,-0.171305,-0.297152,-0.466130,-0.318108,-1.069845,-1.163121,-1.078541,1.793663,-0.870700,0.558739,-0.735092,0.045196,0.058150,-1.153237,-0.341038,0.103445,0.617320,1.117489,2.313443,-0.904136,-1.083621,-1.339735,-0.000007
2,3.821892,-0.484875,-0.821295,-0.297318,-2.636318,-0.841690,-0.658920,-1.691603,1.346548,-0.060072,1.557828,-2.094981,-0.212939,-0.581391,1.692454,0.997538,-0.846012,0.208965,-0.644290,1.799417,2.998226,-1.571814,-2.046079,0.000059
3,-1.329284,1.940988,0.019926,-1.093940,2.190615,-1.291583,-0.667239,-0.474096,-0.487693,1.081803,-0.691977,1.081819,-0.421555,0.101493,2.438611,-1.453767,-1.867994,0.845188,0.519564,-1.980522,0.788272,-0.521639,-1.831544,-0.000010
4,1.489100,0.949028,-1.086673,-0.682466,-1.247095,1.079982,-0.835248,-1.962422,1.035485,1.508746,-0.740490,-3.116952,1.464696,-0.446604,-1.229282,-0.401216,-1.698344,-0.803083,-0.648391,-0.759322,0.925204,-1.194307,-2.633390,0.000098


In [5]:
without_hosp = process_likelihood(data.drop(columns=["HospCt"]))
without_hosp.to_csv('../data/processed/pca_withoutHosp_local_pairwise_multi_var.csv', index=False)
without_hosp.head()

Principal_Component_1+Principal_Component_2+Principal_Component_3+Principal_Component_4+Principal_Component_5+Principal_Component_6+Principal_Component_7+Principal_Component_8+Principal_Component_9+Principal_Component_10+Principal_Component_11+Principal_Component_12+Principal_Component_13+Principal_Component_14+Principal_Component_15+Principal_Component_16+Principal_Component_17+Principal_Component_18+Principal_Component_19+Principal_Component_20+Principal_Component_21
['Principal_Component_1', 'Principal_Component_2', 'Principal_Component_3', 'Principal_Component_4', 'Principal_Component_5', 'Principal_Component_6', 'Principal_Component_7', 'Principal_Component_8', 'Principal_Component_9', 'Principal_Component_10', 'Principal_Component_11', 'Principal_Component_12', 'Principal_Component_13', 'Principal_Component_14', 'Principal_Component_15', 'Principal_Component_16', 'Principal_Component_17', 'Principal_Component_18', 'Principal_Component_19', 'Principal_Component_20', 'Principal_Com

,Principal_Component_1,Principal_Component_2,Principal_Component_3,Principal_Component_4,Principal_Component_5,Principal_Component_6,Principal_Component_7,Principal_Component_8,Principal_Component_9,Principal_Component_10,Principal_Component_11,Principal_Component_12,Principal_Component_13,Principal_Component_14,Principal_Component_15,Principal_Component_16,Principal_Component_17,Principal_Component_18,Principal_Component_19,Principal_Component_20,Principal_Component_21,HospCt,normalized_deaths
0,1.437843,-0.906560,-0.765314,-0.486468,-0.238793,-0.005061,-1.053996,-0.803713,-1.324013,1.281620,-0.411695,0.839990,0.483838,-0.756832,-0.490981,-0.478584,-0.247452,1.045426,2.636560,0.086820,0.416132,-0.727074,-0.000067
1,1.998868,-0.270530,-0.236316,0.481777,-0.212968,0.635795,-1.122611,-0.603409,-2.225429,0.100902,-0.704802,-0.056500,0.187567,-1.321625,-0.223844,0.183408,0.683311,1.091675,2.318314,-0.930500,0.807965,-1.339735,-0.000007
2,3.794394,-0.654015,-0.712654,0.317892,-2.536848,0.568135,-1.712748,0.081125,-1.388164,0.848548,-2.323318,-0.306432,-0.537600,1.001665,1.655809,-0.845292,0.238070,-0.649059,1.855334,2.979466,1.095822,-2.046079,0.000059
3,-1.382241,1.801665,-0.126704,1.105064,2.302861,0.821176,-0.494645,-0.106009,0.707164,0.434876,1.459205,-0.334484,-0.052980,2.341256,-0.608169,-1.515813,1.301855,0.516644,-2.071288,0.795513,0.210544,-1.831544,-0.000010
4,1.433930,0.686854,-1.109566,0.699308,-1.193267,-1.560466,-1.963580,1.733071,-0.050543,-0.091821,-1.615320,1.118544,-0.058938,-2.247785,0.271045,-1.515436,-0.460256,-0.586618,-0.865559,0.911960,0.649495,-2.633390,0.000098
